### 安裝需要的套件
* `Flask`
* `pyngrok`

使用`--quiet`隱藏訊息輸出

In [ ]:
!pip install Flask==3.0.0 pyngrok==7.1.2 --quiet
!pip install line-bot-sdk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.3 MB/s eta 0:00:00
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.5 MB/s eta 0:00:00


### 讀取金鑰放入變數
1. 在程式中**千萬不要**直接顯示金鑰或密碼，在Linux或Windows中設定在環境變數中
2. 如果有安裝conda環境，放入conda的環境變數中
3. 需要預先將金鑰存入Colab的鑰匙圈中
4. 使用Colab提供的`userdata.get()`讀取金鑰

### 指定ngrok使用的通訊埠
1. 此處指定`5000`
2. 在ngrok提供網址後，不需要指定port number，直接使用產生的網站就行

In [ ]:
from google.colab import userdata

ngrok_key = userdata.get('ngrok_key')
line_access_token = userdata.get('line_access_token')
line_secret = userdata.get('line_secret')
port = 5000

### 引入`pyngrok`
目前Colab已禁止使用命令列`ngrok`執行，必須套用`pyngrok`套件，並且取消`ngrok_flask`。

In [ ]:
from pyngrok import ngrok

### 設定ngrok的access token並啟動獲得公網位址
設定access token並且取得公網位址。此值必須放在Line的Webhook值中

In [ ]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://0acc-34-82-18-214.ngrok-free.app'

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                     TextMessage(text=event.message.text)]
            )
        )

@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='位置訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 03:11:32] "POST / HTTP/1.1" 200 -


{"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
{"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"505301892705026261","quoteToken":"7fFcEXV3GJj1E3Uyd7T1qMH-ivHfUNUPLU1oepIs7lCPGD0Z3V4izxAjWBdJZJpv4IMKqpYMZLC8O9PJACEEwPnfGLOcGEu7RhSXN7eo0XQtW-tPBM1uxGFpANph3DCtKvGtoPNvKCLQ9B0bvYkOkg","stickerId":"15","packageId":"1","stickerResourceType":"STATIC","keywords":["silly","Sticking out tongue","No","tease","tongue","moon","Negative"]},"webhookEventId":"01HW9KSK3PK6RP7YHARAC59CZ0","deliveryContext":{"isRedelivery":false},"timestamp":1714014702204,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"e7cd742ddab345bbb6fdb338874917b8","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 03:11:43] "POST / HTTP/1.1" 200 -


{"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"505301900506956357","quoteToken":"qR_C0faK8lAZmhJtPftpP_PZkN0AE5cW4d0blCBPAEIcko9iWkDizmvRyiUuBgQycm7GOImXd5LnlWRq_p17Pezn_QFFAgmPgwSEXf47SuW30z_gQG8RnImpADZoFGlzahkX5mdOYvKFBMv3KeG_ug","text":"A "},"webhookEventId":"01HW9KSQCMN107EZK0S08WMS63","deliveryContext":{"isRedelivery":false},"timestamp":1714014706931,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"948674651dbc472c9041c345bafe6925","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 03:59:39] "POST / HTTP/1.1" 200 -


{"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}


INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:03:13] "GET / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:03:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:03:20] "GET /POST HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:03:22] "GET /POST HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:03:35] "GET / HTTP/1.1" 405 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functio

{"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"505309945751929597","quoteToken":"hCCvcySvTEZ5RrPCn8OTRg-9eCjV19MwnITL0HrnYLWFODFjD8IAGuUlDfBvoPuVX5YX8EHZrEfLKb2toCGGhSK4Fso1wlznjqMXcXed_208zcwEfviVlk0seg2ORh5nhS0B7kkOvvhhgwjTs_eycQ","stickerId":"5","packageId":"1","stickerResourceType":"STATIC","keywords":["moon","Lovely","beauty","Love","Heart","Love you","pretty","Happy"]},"webhookEventId":"01HW9RC2MYWH09FYET0VVY06WF","deliveryContext":{"isRedelivery":false},"timestamp":1714019502353,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"525b0c63e2b84197bb6597db5cfb4961","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [25/Apr/2024 04:31:43] "POST / HTTP/1.1" 200 -


### 執行程式並利用ngrok回訪Colab為Webook


In [ ]:
# 載入flask
from flask import Flask, request

# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

# 載入 json 標準函式庫，處理回傳的資料格式
import json

# 設定app

app = Flask(__name__)

# 設定使用POST方法而非GET方法

@app.route("/", methods=['POST'])

# 定義linebot函式，主要用於回應使用者訊息

def linebot():

    # 取得收到的訊息內容

    body = request.get_data(as_text=True)
    try:
        # json 格式化訊息內容
        json_data = json.loads(body)

        # 輸入channel secret及access token
        line_bot_api = LineBotApi(line_access_token)
        handler = WebhookHandler(line_secret)

        # 加入回傳的 headers
        signature = request.headers['X-Line-Signature']

        # 綁定訊息回傳的相關資訊
        handler.handle(body, signature)

        # 取得 LINE 收到的文字訊息
        # msg = json_data['events'][0]['message']['text']
        ## print(msg)

        # 取得回傳訊息的 Token
        ## tk = json_data['events'][0]['replyToken']

        # 回傳訊息
        ## line_bot_api.reply_message(tk,TextSendMessage(msg))

        # 印出內容
        ## json.dump(json_data)
    except Exception as e:
        # 如果發生錯誤，印出收到的內容
        print(repr(e))
        json.dump(json_data)

    # 驗證 Webhook 使用，不能省略
    return 'OK'

if __name__ == "__main__":
  app.run(port=port)

### 回傳使用者傳來的貼圖
1. 重新啟動ngrok獲得新的Webhook
2. 取得貼圖的Package id及Sticker id，然後回傳

In [ ]:
ngrok.connect(port).public_url

In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler

# 載入 StickerSendMessage 模組
from linebot.models import StickerSendMessage
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
        line_bot_api = LineBotApi(line_access_token)
        handler = WebhookHandler(line_secret)
        signature = request.headers['X-Line-Signature']

        handler.handle(body, signature)

        tk = json_data['events'][0]['replyToken']

        # 取得 stickerId
        stickerId = json_data['events'][0]['message']['stickerId']

        # 取得 packageId
        packageId = json_data['events'][0]['message']['packageId']

        # 設定要回傳的表情貼圖
        sticker_message = StickerSendMessage(sticker_id=stickerId, package_id=packageId)

        # 回傳貼圖
        line_bot_api.reply_message(tk,sticker_message)
        json.dump(json_data)
    except:
        print(body)
    return 'OK'

if __name__ == "__main__":
    app.run(port=port)

### 回覆影像及影片
1. 重新啟動ngrok獲得Webhook
2. 建立一個`reply_img()`函數，去尋找網頁上是否有這個圖(來自`url`)


In [ ]:
ngrok.connect(port).public_url

In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler

# 載入 ImageSendMessage 模組
from linebot.models import TextSendMessage, ImageSendMessage
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])

def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
        line_bot_api = LineBotApi(line_access_token)
        handler = WebhookHandler(line_secret)
        signature = request.headers['X-Line-Signature']

        handler.handle(body, signature)

        tk = json_data['events'][0]['replyToken']
        msg = json_data['events'][0]['message']['text']

        # 取得對應的圖片，如果沒有取得，會是 False
        img_url = reply_img(msg)
        if img_url:
            # 如果有圖片網址，回傳圖片
            img_message = ImageSendMessage(original_content_url=img_url, preview_image_url=img_url)
            line_bot_api.reply_message(tk,img_message)
        else:
            # 如果是 False，回傳文字
            text_message = TextSendMessage(text='找不到圖片')
            line_bot_api.reply_message(tk,text_message)
        # 印出訊息本體
        json.dump(json_data)
    except:
        print(body)
    return 'OK'
# 建立回覆圖片的函式
def reply_img(text):
    # 文字對應圖片網址的字典
    img = {
        '皮卡丘':'https://upload.wikimedia.org/wikipedia/en/a/a6/Pok%C3%A9mon_Pikachu_art.png',
        '傑尼龜':'https://upload.wikimedia.org/wikipedia/en/5/59/Pok%C3%A9mon_Squirtle_art.png',
        '東吳': 'https://www-ch.scu.edu.tw/assets/images/logo_scu_all.png'
    }
    if text in img:
      return img[text]
    else:
      # 如果找不到對應的圖片，回傳 False
      return False

if __name__ == "__main__":
    app.run(port=port)

### 傳送位置資訊


In [ ]:
ngrok.connect(port).public_url

In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler

# 載入 LocationSendMessage 模組
from linebot.models import TextSendMessage, LocationSendMessage
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])

def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
        line_bot_api = LineBotApi(line_access_token)
        handler = WebhookHandler(line_secret)
        signature = request.headers['X-Line-Signature']

        handler.handle(body, signature)

        tk = json_data['events'][0]['replyToken']
        msg = json_data['events'][0]['message']['text']
        location_dect = reply_location(msg)

        # 取得對應的地址，如果沒有取得，會是 False
        if location_dect:
            # 如果有地點資訊，回傳地點
            location_message = LocationSendMessage(title=location_dect['title'],
                                  address=location_dect['address'],
                                  latitude=location_dect['latitude'],
                                  longitude=location_dect['longitude'])
            line_bot_api.reply_message(tk,location_message)
        else:
            # 如果是 False，回傳文字
            text_message = TextSendMessage(text='找不到相關地點')
            line_bot_api.reply_message(tk,text_message)
        json.dump(json_data)
    except:
        print(body)
    return 'OK'
# 建立回覆地點的函式
def reply_location(text):
    # 建立地點與文字對應的字典
    location = {
        '101':{
            'title':'台北 101',
            'address':'110台北市信義區信義路五段7號',
            'latitude':'25.034095712145003',
            'longitude':'121.56489941996108'
        },
        '東吳':{
            'title':'東吳大學校本部',
            'address':'111台北市士林區臨溪路70號',
            'latitude':'25.0945684',
            'longitude':'121.5428445'
        },
        '總統府':{
            'title':'總統府',
            'address':'100台北市中正區重慶南路一段122號',
            'latitude':'25.040319874750914',
            'longitude':'121.51162883484746'
        }
    }
    if text in location:
      return location[text]
    else:
      # 如果找不到對應的地點，回傳 False
      return False

if __name__ == "__main__":
    app.run(port=port)

### 綜合以上所有功能

In [ ]:
ngrok.connect(port).public_url

In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler

# 載入 LocationSendMessage 模組
from linebot.models import TextSendMessage, LocationSendMessage
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])

def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
        # access token
        # secret

        line_bot_api = LineBotApi('line_access_token')
        handler = WebhookHandler('line_secret')
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)

        # 取得 reply token
        tk = json_data['events'][0]['replyToken']

        # 取得 message 的類型
        tp = json_data['events'][0]['message']['type']
        print("!!!!", tp)
        if tp == 'text':

            # 如果是文字類型的訊息
            # 取出文字並對應到 reply_msg 的函式
            msg = reply_msg(json_data['events'][0]['message']['text'])
            if msg[0] == 'text':
                # 如果要回傳的訊息是 text，使用 TextSendMessage 方法
                line_bot_api.reply_message(tk,TextSendMessage(text=msg[1]))
            if msg[0] == 'location':
                # 如果要回傳的訊息是 location，使用 LocationSendMessage 方法
                line_bot_api.reply_message(tk,LocationSendMessage(title=msg[1]['title'],
                                                address=msg[1]['address'],
                                                latitude=msg[1]['latitude'],
                                                longitude=msg[1]['longitude']))
            if msg[0] == 'image':
                # 如果要回傳的訊息是 image，使用 ImageSendMessage 方法
                line_bot_api.reply_message(tk,ImageSendMessage(original_content_url=msg[1],
                                                    preview_image_url=msg[1]))
        if tp == 'sticker':
            # 如果收到的訊息是表情貼圖
            stickerId = json_data['events'][0]['message']['stickerId'] # 取得 stickerId
            packageId = json_data['events'][0]['message']['packageId'] # 取得 packageId

            # 使用 StickerSendMessage 方法回傳同樣的表情貼圖
            line_bot_api.reply_message(tk,StickerSendMessage(sticker_id=stickerId, package_id=packageId))
        if tp == 'location':

            # 如果是收到的訊息是地點資訊
            line_bot_api.reply_message(tk,TextSendMessage(text='地點'))
        if tp == 'image':

            # 如果是收到的訊息是圖片
            line_bot_api.reply_message(tk,TextSendMessage(text='圖片'))
        if tp == 'audio':

            # 如果是收到的訊息是聲音
            line_bot_api.reply_message(tk,TextSendMessage(text='聲音'))
        if tp == 'video':
            # 如果是收到的訊息是影片
            line_bot_api.reply_message(tk,TextSendMessage(text='影片'))
    except:
        print('error', body)
    return 'OK'
# 定義回覆訊息的函式
def reply_msg(text):
    # 客製化回覆文字
    msg_dict = {
        'hi':'Hi! 你好',
        'hello':'Hello World!',
        '你好':'你好',
        'help':'有什麼要幫忙的嗎？'
    }
    # 如果出現特定地點，提供地點資訊
    local_dict = {
        '東吳大學':{
            'title':'東吳大學校本部',
            'address':'111台北市士林區臨溪路70號',
            'latitude':'25.0945684',
            'longitude':'121.5428445'
        }
    }
    # 如果出現特定圖片文字，提供圖片網址
    img_dict = {
        '東吳': 'https://www-ch.scu.edu.tw/assets/images/logo_scu_all.png',
        '皮卡丘':'https://upload.wikimedia.org/wikipedia/en/a/a6/Pok%C3%A9mon_Pikachu_art.png',
        '傑尼龜':'https://upload.wikimedia.org/wikipedia/en/5/59/Pok%C3%A9mon_Squirtle_art.png'
    }
    # 預設回覆的文字就是收到的訊息
    reply_msg_content = ['text',text]
    if text in msg_dict:
        reply_msg_content = ['text',msg_dict[text.lower()]]
    if text in local_dict:
        reply_msg_content = ['location',local_dict[text.lower()]]
    if text in img_dict:
        reply_msg_content = ['image',img_dict[text.lower()]]
    return reply_msg_content

if __name__ == "__main__":
    app.run(port=port)

In [ ]:
# from flask import Flask

# app = Flask(__name__)

# @app.route("/<name>")
# def hello(name):
#     return f"<h1>Hello {name}</h1>"

# if __name__ == '__main__':
#     app.run(port = port)